# Лабораторна робота №2: "Наука про дані: підготовчий етап"
### *виконала студентка групи ФБ-33 Журавльова Марія*

**Мета роботи:** ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції.

**Постановка задачі:**
- Проаналізувати часові ряди глобальних продуктів по оцінці вегетаційного здоров’я VHI (vegetation health index) http://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/index.php, який надається Національною адміністрацією океанів та атмосфери США NOAA (http://www.noaa.gov/);
- Виявити особливості ходу індексу впродовж вегетаційного періоду (вересень року-попередника – липень поточного року) в розрізі областей України;
- Додаткові завдання по фільтрації даних від викладача, який веде практику.

**Хід виконання роботи:**
1. Створити віртуальне середовище (venv) в якому будуть встановлені всі необхідні бібліотеки та налаштування для даної лабораторної роботи.
Для цього використовую команди:
```
pip install virtualenv
python -m virtualenv vhi_env
vhi_env\Scripts\activate
```

Необхідні бібліотеки:

In [1]:
import urllib.request
import pandas as pd
import os
from datetime import datetime
import re
from tabulate import tabulate
print("Setup Complete")

Setup Complete


2. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних.

In [2]:
country = "UKR"
year_1 = 1981
year_2 = 2024
type_data = "Mean"
dir_data = r"C:/Users/User/vhi_venv/data_csv"

if not os.path.exists(dir_data):
    os.makedirs(dir_data)
    print(f"Директорія {dir_data} створена.")

for filename in os.listdir(dir_data):
    filepath = os.path.join(dir_data, filename)
    if filename.endswith(".csv"):
        os.remove(filepath)
print("Директорія очищена від старих файлів.")

def download_csv(province_id):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country={country}&provinceID={province_id}&year1={year_1}&year2={year_2}&type={type_data}"
    
    try:
        with urllib.request.urlopen(url) as response:
            text = response.read()
    except urllib.error.URLError as e:
        print(f"!!! Помилка завантаження для області {province_id}: {e}")
        return

    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"VHI_province_{province_id}_{timestamp}.csv"
    filepath = os.path.join(dir_data, filename)

    try:
        with open(filepath, 'wb') as out_file:
            out_file.write(text)
        print(f"Файл {filename} завантажено успішно.")
    except IOError as e:
        print(f"Помилка запису файлу {filename}: {e}")
        
for province in range(1, 28):
    download_csv(province)

Директорія очищена від старих файлів.
Файл VHI_province_1_20250314160940.csv завантажено успішно.
Файл VHI_province_2_20250314160942.csv завантажено успішно.
Файл VHI_province_3_20250314160943.csv завантажено успішно.
Файл VHI_province_4_20250314160944.csv завантажено успішно.
Файл VHI_province_5_20250314160945.csv завантажено успішно.
Файл VHI_province_6_20250314160946.csv завантажено успішно.
Файл VHI_province_7_20250314160947.csv завантажено успішно.
Файл VHI_province_8_20250314160948.csv завантажено успішно.
Файл VHI_province_9_20250314160949.csv завантажено успішно.
Файл VHI_province_10_20250314160950.csv завантажено успішно.
Файл VHI_province_11_20250314160951.csv завантажено успішно.
Файл VHI_province_12_20250314160952.csv завантажено успішно.
Файл VHI_province_13_20250314160953.csv завантажено успішно.
Файл VHI_province_14_20250314160954.csv завантажено успішно.
Файл VHI_province_15_20250314160955.csv завантажено успішно.
Файл VHI_province_16_20250314160956.csv завантажено успі

3. Зчитати завантажені текстові файли у фрейм (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли.

In [3]:
def remove_html_tags(text):
    if isinstance(text, str):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text).strip()
    return text

def load_vhi_data(directory):
    files = [f for f in os.listdir(directory) if f.endswith(".csv")]

    if not files:
        print("!!! Помилка: У директорії немає CSV-файлів.")
        return None

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_data = []

    for file in files:
        filepath = os.path.join(directory, file)
        
        parts = file.split('_')
        region_id = parts[2] if len(parts) > 2 else None
        
        df = pd.read_csv(filepath, header=1, names=headers, converters={'Year': remove_html_tags})
        df = df.drop(columns=['empty'], errors='ignore')
        df.replace(-1, pd.NA, inplace=True)
        df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
        df.insert(0, "Region_ID", region_id)
        all_data.append(df)

    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df.sort_values(by=["Region_ID", "Year", "Week"]).reset_index(drop=True)

    return final_df

def print_dataframe(df, rows=10):
    print(tabulate(df.head(rows), headers='keys', tablefmt="fancy_grid", showindex=False))

directory_path = r"C:\Users\User\vhi_venv\data_csv"

df_vhi = load_vhi_data(directory_path)

if df_vhi is not None:
    print_dataframe(df_vhi, rows=10)


╒═════════════╤════════╤════════╤═══════╤════════╤═══════╤═══════╤═══════╕
│   Region_ID │   Year │   Week │   SMN │    SMT │   VCI │   TCI │   VHI │
╞═════════════╪════════╪════════╪═══════╪════════╪═══════╪═══════╪═══════╡
│           1 │   1982 │      1 │ 0.053 │ 260.31 │ 45.01 │ 39.46 │ 42.23 │
├─────────────┼────────┼────────┼───────┼────────┼───────┼───────┼───────┤
│           1 │   1982 │      2 │ 0.054 │ 262.29 │ 46.83 │ 31.75 │ 39.29 │
├─────────────┼────────┼────────┼───────┼────────┼───────┼───────┼───────┤
│           1 │   1982 │      3 │ 0.055 │ 263.82 │ 48.13 │ 27.24 │ 37.68 │
├─────────────┼────────┼────────┼───────┼────────┼───────┼───────┼───────┤
│           1 │   1982 │      4 │ 0.053 │ 265.33 │ 46.09 │ 23.91 │ 35    │
├─────────────┼────────┼────────┼───────┼────────┼───────┼───────┼───────┤
│           1 │   1982 │      5 │ 0.05  │ 265.66 │ 41.46 │ 26.65 │ 34.06 │
├─────────────┼────────┼────────┼───────┼────────┼───────┼───────┼───────┤
│           1 │   1982 │ 

4. Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA (за англійською абеткою) на наступні, за українською (виключно старі індекси на нові):

| № області | Назва            | №області | Назва             |
|-----------|------------------|----------|-------------------|
| 1         | Вінницька        | 13       | Миколаївська      |
| 2         | Волинська        | 14       | Одеська           |
| 3         | Дніпропетровська | 15       | Полтавська        |
| 4         | Донецька         | 16       | Рівенська        |
| 5         | Житомирська      | 17       | Сумська           |
| 6         | Закарпатська     | 18       | Тернопільська     |
| 7         | Запорізька       | 19       | Харківська        |
| 8         | Івано-Франківська| 20       | Херсонська        |
| 9         | Київська         | 21       | Хмельницька       |
| 10        | Кіровоградська   | 22       | Черкаська         |
| 11        | Луганська        | 23       | Чернівецька       |
| 12        | Львівська        | 24       | Чернігівська      |
|           |                  | 25       | Республіка Крим   |


Для цього виводжу № області та її назву англійською для розуміння що на що змінювати.

In [4]:
def region_info(directory):
    region_info = []

    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        if file_name.endswith('.csv'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

                matches = re.findall(r"= (\d+): ([\w'' '-]+),", content)
                
                if matches:
                    region_info.extend(matches)

    return region_info

directory_path = r"C:\Users\User\vhi_venv\data_csv"

regions = region_info(directory_path)

if regions:
    for region_id, region_name in regions:
        print(f"{region_id:<3} | {region_name}")
else:
    print("Області не знайдені у файлах!")

10  | Khmel'nyts'kyy
11  | Kiev
12  | Kiev City
13  | Kirovohrad
14  | Luhans'k
15  | L'viv
16  | Mykolayiv
17  | Odessa
18  | Poltava
19  | Rivne
1   | Cherkasy
20  | Sevastopol'
21  | Sumy
22  | Ternopil'
23  | Transcarpathia
24  | Vinnytsya
25  | Volyn
26  | Zaporizhzhya
27  | Zhytomyr
2   | Chernihiv
3   | Chernivtsi
4   | Crimea
5   | Dnipropetrovs'k
6   | Donets'k
7   | Ivano-Frankivs'k
8   | Kharkiv
9   | Kherson


In [5]:
def replace_region_ids(data_frames):

    region_mapping = {
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9,
        13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 
        23: 6, 24: 1, 25: 2, 26: 7, 27: 5
    }

    data_frames_cp = data_frames.copy()

    if "Region_ID" in data_frames_cp.columns:
        data_frames_cp["Region_ID"] = pd.to_numeric(data_frames_cp["Region_ID"], errors="coerce").astype("Int64")
        data_frames_cp["Year"] = pd.to_numeric(data_frames_cp["Year"], errors="coerce").astype("Int64")
        data_frames_cp["VHI"] = pd.to_numeric(data_frames_cp["VHI"], errors="coerce")
        
        data_frames_cp.loc[:, "Region_ID"] = data_frames_cp["Region_ID"].replace(region_mapping)
        print("Індекси областей успішно оновлені!")
    else:
        print("!!! Помилка: у DataFrame немає колонки 'Region_ID'")

    return data_frames_cp

directory_path = r"C:\Users\User\vhi_venv\data_csv"
df_vhi = load_vhi_data(directory_path)
df_vhi_replace = replace_region_ids(df_vhi)

Індекси областей успішно оновлені!


5. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік;

In [6]:
def get_vhi_for_region_year(df, region_id, year):
    filtered_df = df[(df["Region_ID"] == region_id) & (df["Year"] == year)]

    if filtered_df.empty:
        print("Даних для вибраного року та області немає!")
        return None
        
    return filtered_df[["Year", "Week", "VHI"]]

region_id = 4
year = 2006

get_vhi_for_region_year(df_vhi_replace, region_id, year)

,Year,Week,VHI
52699,2006,1.0,49.64
52700,2006,2.0,53.94
52701,2006,3.0,53.77
52702,2006,4.0,54.25
52703,2006,5.0,56.88
52704,2006,6.0,57.52
52705,2006,7.0,56.58
52706,2006,8.0,56.60
52707,2006,9.0,56.38
52708,2006,10.0,56.58


- Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;

In [7]:
def analyze_vhi(df, region_ids, start_year, end_year):
    filtered_df = df[(df["Region_ID"].isin(region_ids)) & (df["Year"].between(start_year, end_year))]

    if filtered_df.empty:
        print("Даних для вибраних областей і років немає!")
        return None

    summary = (filtered_df.groupby(["Region_ID", "Year"])["VHI"].agg(["min", "max", "mean", "median"]))
    return summary

region_id = [1, 5, 10]
start_year = 2015
end_year = 2020

analyze_vhi(df_vhi_replace, region_id, start_year, end_year)

min    max       mean  median
Region_ID Year                                 
1         2015  19.94  50.72  38.936538  41.915
          2016  31.59  68.76  49.849231  48.385
          2017  38.22  58.19  48.114231  48.290
          2018  36.64  69.48  50.458077  48.700
          2019  20.26  67.33  45.275000  44.830
          2020  34.48  64.12  45.911538  44.230
5         2015  32.48  52.27  42.938846  43.445
          2016  37.18  56.23  47.642692  47.865
          2017  39.84  61.74  49.271346  46.755
          2018  39.32  60.82  49.507500  49.100
          2019  30.24  60.57  45.644615  47.290
          2020  35.05  58.33  44.544231  43.660
10        2015  26.05  71.33  43.449231  39.475
          2016  33.77  76.98  50.710577  45.585
          2017  31.26  60.97  44.136923  44.225
          2018  34.04  66.75  47.756923  47.415
          2019  22.08  73.05  44.900962  41.185
          2020  22.73  63.63  41.995577  40.930

- Ряд VHI за вказаний діапазон років для вказаних областей;

In [12]:
def get_vhi_for_regions_years(df, region_ids, start_year, end_year):
    filtered_df = df[(df["Region_ID"].isin(region_ids)) & (df["Year"].between(start_year, end_year))]

    if filtered_df.empty:
        print("Даних для вибраного діапазону років і областей немає!")
        return None

    return filtered_df[["Region_ID", "Year", "Week", "VHI"]]

region_ids = [5, 10]
start_year = 2005
end_year = 2015
get_vhi_for_regions_years(df_vhi_replace, region_ids, start_year, end_year)

,Region_ID,Year,Week,VHI
10144,10,2005,1.0,47.01
10145,10,2005,2.0,46.40
10146,10,2005,3.0,45.85
10147,10,2005,4.0,46.40
10148,10,2005,5.0,46.73
...,...,...,...,...
44266,5,2015,48.0,42.39
44267,5,2015,49.0,43.36
44268,5,2015,50.0,45.11
44269,5,2015,51.0,46.13


- Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI.

In [22]:
def find_drought_years(df, threshold=15, min_regions=5):
    drought_data = df[df["VHI"] < threshold]
    drought_summary = drought_data.groupby("Year")["Region_ID"].nunique()
    severe_drought_years = drought_summary[drought_summary >= min_regions].index

    results = []
    for year in severe_drought_years:
        affected_regions = df[(df["Year"] == year) & (df["VHI"] < threshold)][["Region_ID", "VHI"]]
        
        affected_summary = affected_regions.groupby("Region_ID")["VHI"].mean().reset_index()

        region_vhi_data = [
            f"Область {row.Region_ID} (VHI: {row.VHI:.2f})"
            for row in affected_summary.itertuples()
        ]

        formatted_regions = "\n".join(region_vhi_data)

        year_data = {
            "Year": year,
            "Affected_Regions": formatted_regions
        }
        results.append(year_data)

    return results

drought_years = find_drought_years(df_vhi)
print(tabulate(drought_years, headers="keys", tablefmt="fancy_grid"))

╒════════╤═════════════════════════╕
│   Year │ Affected_Regions        │
╞════════╪═════════════════════════╡
│   2000 │ Область 1 (VHI: 12.41)  │
│        │ Область 11 (VHI: 12.26) │
│        │ Область 12 (VHI: 9.53)  │
│        │ Область 20 (VHI: 10.83) │
│        │ Область 24 (VHI: 12.21) │
│        │ Область 8 (VHI: 11.46)  │
├────────┼─────────────────────────┤
│   2007 │ Область 16 (VHI: 7.90)  │
│        │ Область 17 (VHI: 8.11)  │
│        │ Область 26 (VHI: 12.64) │
│        │ Область 4 (VHI: 14.01)  │
│        │ Область 9 (VHI: 13.29)  │
╘════════╧═════════════════════════╛
